## Importing Required Libraries

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Data Pre-processing

### Training Image Preprocessing

In [ ]:
training_dataset = tf.keras.utils.image_dataset_from_directory(
    'Train',
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = None,
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (128, 128),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = 'bilinear',
    follow_links = False,
    crop_to_aspect_ratio = False,
    pad_to_aspect_ratio = False,
    verbose = True,
)

### Validation Image Preprocessing

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    'Valid',
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = None,
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (128, 128),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = 'bilinear',
    follow_links = False,
    crop_to_aspect_ratio = False,
    pad_to_aspect_ratio = False,
    verbose = True,
)

In [ ]:
training_dataset

## Building Model

In [25]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

In [26]:
model = Sequential()

#### Building a Convolution Layer

In [27]:
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[128,128,3]))
model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [28]:
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [29]:
model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [30]:
model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [31]:
model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

In [32]:
model.add(Dropout(0.25))

In [33]:
model.add(Flatten())

In [34]:
model.add(Dense(units=1500, activation='relu'))

In [35]:
model.add(Dropout(0.4))

In [36]:
#Output Layer

model.add(Dense(units=38, activation='softmax'))

#### Compiling Model

In [37]:
model.compile(tf.keras.optimizers.Adam(
    learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

#### Building the Model

We need to make sure the model is not going through overshooting, overfitting or underfitting.
To avoid Overshooting:
1. Choose small learning rate. Default is 0.001.
2. Increase number of neurons to prevent underfitting.
3. Add more convolution layer to extract more features from images. There may be a possibility that the model is unable to capture relevant feature or model is confused due to lack of features.

### Training Model

In [ ]:
training_history = model.fit(x=training_dataset, validation_data = validation_dataset, epochs = 10)

## Model Evaluation

In [ ]:
#Model Evaluation on Training set
training_loss, training_accuracy = model.evaluate(training_dataset)

In [ ]:
print(training_loss, training_accuracy)

In [ ]:
#Model Evaluation on Validation Set
validation_loss, validation_accuracy = model.evaluate(validation_dataset)

In [ ]:
print(validation_loss, validation_accuracy)

## Saving the Model

In [ ]:
model.save("trained_model.keras") #for less memory usage

In [ ]:
training_history.history

## Recording History in JSON

In [ ]:
import json
with open("training_hist.json", "w"):
    json.dump(training_history.history, f)

## Accuracy Visualization

In [ ]:
epochs = [i for i in range(1, 11)]
plt.plot(epochs, training_history.history["accuracy"], color = "green", label = "Training Accuracy")
plt.plot(epochs, training_history.history["val_accuracy"], color = "red", label = "Validation Accuracy")
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy Result")
plt.title("Visualization of Accuracy Result")
plt.legend()
plt.show()